In [1]:
import pandas as pd
import pingouin as pg
from statsmodels.stats.multitest import multipletests
import numpy as np
from scipy.stats import *

In [2]:
# Delete genes if half or more of them have a 0 value and if not turn to NaN values
def delete_0s(df):
    deleted_cols = df.columns[(df == 0).sum() >= len(df)/2]
    df = df.drop(columns = deleted_cols)
    print(f'Columnas eliminadas {deleted_cols}')if len(deleted_cols) else None
    # df = df.replace(0, np.nan)
    return df
    
def delete_outliers(df):
    
    means = df.mean()
    std_devs = df.std()
    
    for column in df.columns:
        mean = means[column]
        std_dev = std_devs[column]     
        
        # Turn to 0 outliers, an outlier is over/below the mean+2*desv
        df.loc[(df[column] > mean + 2 * std_dev) | (df[column] < mean - 2 * std_dev), column] = np.nan
        #print(column)

def metrics(df_female, df_male):
    p_values_ttest, normality_f, normality_m, homoscedasticity, p_values_anova = [], [], [], [], []
    
    for column in df_female.columns:
        female_values = df_female[column].dropna().tolist()
        male_values = df_male[column].dropna().tolist()
        
        # t- student P-value
        # result = pg.ttest(female_values.dropna().tolist(), male_values.dropna().tolist(), correction=False)
        result = pg.ttest(female_values, male_values, correction=False)
        p_value = result['p-val'].iloc[0] 
        p_values_ttest.append(p_value)

        # normality by groups
        _, p_value = shapiro(female_values)
        normality_f.append(p_value)
        _, p_value = shapiro(male_values)
        normality_m.append(p_value)

        # homocedasticity
        _, p_value = levene(female_values, male_values)
        homoscedasticity.append(p_value)

        # ANOVA
        _, p_value = f_oneway(female_values, male_values)
        p_values_anova.append(p_value)
    

    # Adjusted p-values
    # adjusted_p_values_ttest = multipletests(p_values_ttest, method='fdr_bh')[1]
    # adjusted_p_values_anova = multipletests(p_values_anova, method='fdr_bh')[1]
    
    result_df = pd.DataFrame({
        'Normality_female': normality_f,
        'Normality_male': normality_m,
        'Homocedasticity': homoscedasticity,
        'P-value ANOVA': p_values_anova,
        # 'Adjusted_p-value ANOVA': adjusted_p_values_anova,
        'P-value T-test': p_values_ttest,
        # 'Adjusted_p-value T-test': adjusted_p_values_ttest,
        'Fold_change': df_male.mean(axis=0)/df_female.mean(axis=0),
        'Mean_female': df_female.mean(axis=0),
        'Mean_male': df_male.mean(axis=0),
        'Std_female': df_female.std(axis=0),
        'Std_male': df_male.std(axis=0),
    })
    result_df = result_df.transpose()
    return(result_df)

def normalize_columns(df):
    for col in df.columns:
        df[col] = np.log10(df[col])
        # mask = ~df[col].isnull()
        # transformed_values, _ = boxcox(df[col][mask]) 
        # df[col][mask] = transformed_values
    return df

In [8]:
df= pd.read_excel('C:/Users/User/Documents/Universidad/6º Carrera/Practicas/Metabolomic/Data/2024-03-22_Conc_Plate4_HUMORAC_curated.xlsx', sheet_name='Data_4')
demographic_data = pd.read_excel('C:/Users/User/Documents/Universidad/6º Carrera/Practicas/Metabolomic/Data/2024-03-22_Conc_Plate4_HUMORAC_curated.xlsx', sheet_name='Demographic_data_2')
df = df.replace(0, np.nan)
df = df.loc[df['Disease'] == 'C']

# Merge both df with the sex info of each sample
demographic_data = demographic_data[['Sample', 'Sex']]
df = pd.merge(demographic_data, df, how='inner', on='Sample')

df.set_index('Sample', inplace=True)
df = df.drop(columns=['Type', 'Disease'])
# print(df)

# Split by sex
df_female = df.loc[df['Sex'] == 1]
df_female = df_female.drop(columns=['Sex'])
df_male = df.loc[df['Sex'] == 0]
df_male = df_male.drop(columns=['Sex'])
print( df_female, df_male)

# Delete outliers
delete_outliers(df_female)
delete_outliers(df_male)

# Delete mewtabolites if half or more of them have a 0 value and if not turn to NaN values
df_female = delete_0s(df_female)
df_male = delete_0s(df_male)
print( df_female, df_male)

# Metabolites' names without outliers
col_names_no_OLs = df_female.columns.intersection(df_male.columns).tolist()

print(len(col_names_no_OLs))

# Delete columns removed from the other df
df_female_no_OLs = df_female.loc[:, df_female.columns.isin(col_names_no_OLs)]
df_male_no_OLs = df_male.loc[:, df_male.columns.isin(col_names_no_OLs)]

# Normalize
df_female_no_OLs = normalize_columns(df_female_no_OLs.copy())
df_male_no_OLs = normalize_columns(df_male_no_OLs.copy())
# print(df_control_no_OLs, df_treated_no_OLs)

# Join dfs
df_no_OLs =  pd.concat([df_female_no_OLs, df_male_no_OLs])
# df_no_OLs.to_excel('data_no_OLs.xlsx')
# print(df_no_OLs)

# Get metrics
metrics = metrics(df_female_no_OLs,df_male_no_OLs)
results = pd.concat([df_no_OLs, metrics])
# print(results)


# col_names = results.columns[(results.loc['Fold_change'] >= 1.3) | (results.loc['Fold_change'] <= 1/1.3) ]
# col_names = results.columns[(results.loc['Fold_change'] >= 1.15) | (results.loc['Fold_change'] <= 1/1.15) ]
col_names = results.columns
print(len(col_names))

results_filtered = results[col_names]
p_values_anova = results_filtered.loc['P-value ANOVA'].tolist()
p_values_ttest = results_filtered.loc['P-value T-test'].tolist()
adjusted_p_values_anova = multipletests(p_values_anova, method='fdr_bh')[1]
adjusted_p_values_ttest = multipletests(p_values_ttest, method='fdr_bh')[1]

results_filtered = results_filtered.copy()
results_filtered.loc['Adjusted_p-value ANOVA'] = adjusted_p_values_anova
results_filtered.loc['Adjusted_p-value T-test'] = adjusted_p_values_ttest

print(results_filtered)

# results_filtered.to_excel('results_CTRL.xlsx')
#'''

               C0        C2        C3  C3-DC (C4-OH)        C4      C4:1  \
Sample                                                                     
c1       6.744444  1.025556  0.124778       0.005667  0.037556  0.010778   
c3       7.733333  1.170000  0.226667       0.007000  0.063333  0.008667   
c4       7.466667  0.836667  0.138333       0.006333  0.058667  0.010333   
c5       7.800000  0.863333  0.185667       0.005333  0.045333  0.008667   
c6       8.466667  1.036667  0.150333       0.008333  0.052333  0.009667   
c7       3.400000  0.380000  0.070333       0.003667  0.023667  0.006000   
c8       9.533333  1.016667  0.168000       0.007000  0.071333  0.009333   
c11      5.727273  0.854545  0.140909       0.010000  0.045909  0.017273   
RF_138   8.450000  1.590000  0.140250       0.007000  0.044250  0.007000   
RF_141  12.725000  2.075000  0.427500       0.010250  0.112500  0.008250   
RF_143   8.625000  1.862500  0.229250       0.008250  0.051750  0.007000   
RF_144   9.9

"\n# Metabolites' names without outliers\ncol_names_no_OLs = df_female.columns.intersection(df_male.columns).tolist()\n\nprint(len(col_names_no_OLs))\n\n# Delete columns removed from the other df\ndf_female_no_OLs = df_female.loc[:, df_female.columns.isin(col_names_no_OLs)]\ndf_male_no_OLs = df_male.loc[:, df_male.columns.isin(col_names_no_OLs)]\n\n# Normalize\ndf_female_no_OLs = normalize_columns(df_female_no_OLs.copy())\ndf_male_no_OLs = normalize_columns(df_male_no_OLs.copy())\n# print(df_control_no_OLs, df_treated_no_OLs)\n\n# Join dfs\ndf_no_OLs =  pd.concat([df_female_no_OLs, df_male_no_OLs])\n# df_no_OLs.to_excel('data_no_OLs.xlsx')\n# print(df_no_OLs)\n\n# Get metrics\nmetrics = metrics(df_female_no_OLs,df_male_no_OLs)\nresults = pd.concat([df_no_OLs, metrics])\n# print(results)\n\n\n# col_names = results.columns[(results.loc['Fold_change'] >= 1.3) | (results.loc['Fold_change'] <= 1/1.3) ]\n# col_names = results.columns[(results.loc['Fold_change'] >= 1.15) | (results.loc['Fold_

In [54]:
df= pd.read_excel('C:/Users/User/Documents/Universidad/6º Carrera/Practicas/Metabolomic/Data/2024-03-22_Conc_Plate4_HUMORAC_curated.xlsx', sheet_name='Data_3')
df.set_index('Sample', inplace=True)
old = df.loc[df['Type'] == 'VIEJO']
new = df.loc[df['Type'] == 'NUEVO']

new_control = new.loc[df['Disease'].str.startswith('C')]
new_control = new_control.drop(columns=['Disease', 'Type'])
new_treated = new.loc[df['Disease'].str.startswith('P')]
new_treated = new_treated.drop(columns=['Disease', 'Type'])
old_control = old.loc[df['Disease'].str.startswith('C')]
old_control = old.drop(columns=['Disease', 'Type'])
old_treated = old.loc[df['Disease'].str.startswith('P')]
old_treated = old.drop(columns=['Disease', 'Type'])


result_df = pd.DataFrame({
        'Mean_control_new': new_control.mean(axis=0),
        'Mean_control_old': old_control.mean(axis=0),
        'Mean_PD_new': new_treated.mean(axis=0),
        'Mean_PD_old': old_treated.mean(axis=0),
        'Std_control_new': new_control.std(axis=0),
        'Std_control_old': old_control.std(axis=0),
        'Std_PD_new': new_treated.std(axis=0),
        'Std_PD_old': old_treated.std(axis=0),
    })
result_df = result_df.transpose()
result_df.to_excel('Means_desv.xlsx')
